## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from folium.plugins import MarkerCluster

import folium

from src.data_aquisition.overpass import get_pois_near_path

## 1. Take hard-coded route as reference

There are some standard formats for saving geo routes:
- GeoJSON (commonly used with web libraries like leaflet and google maps)
- GPX (supported by GPS device and mobile apps, has elevation)
- KML (used by google, good for integration with google maps)
- csv (simple approach)
- SHP (popular for GIS)

In [3]:
kml_file_path = 'path.kml'
gdf = gpd.read_file(kml_file_path, driver='KML')

route_geom = gdf.geometry.iloc[0]

first_point = route_geom.coords[0][1], route_geom.coords[0][0]
m = folium.Map(location=[*first_point], zoom_start=15)

coords_lat_lon = [(point[1], point[0]) for point in route_geom.coords]  # convert from (lon, lat) to (lat, lon)
folium.PolyLine(coords_lat_lon, color="blue", weight=2.5, opacity=1).add_to(m)


# add buffer
buffer_distance = 0.0002  # 20m
route_buffer = route_geom.buffer(buffer_distance)

buffer_coords = list(route_buffer.exterior.coords)  # Get the coordinates of the buffer polygon
folium.Polygon(locations=[(point[1], point[0]) for point in buffer_coords], color='green', fill=True, fill_opacity=0.2).add_to(m)


m

### 2. Find objects along the route and get their info

#### 2.1 Specific info

How to find the info?
1. Create a buffer zone around the route
2. Use a geospatial query to find objects within this buffer
3. Retrieve data about the found objects

We can use 
1. OpenStreetMap via Overpass API (free)
2. Google Places API (advantages: ratings, photos, more places, but not free)
3. Foursquare API

In [28]:
tags = ["historic", "tourism"]

route = LineString(route_geom.coords)
pois_near_path = get_pois_near_path(route, tags, buffer_distance=20, max_distance=20)
pois_near_path = [el for el in pois_near_path if el['lat'] is not None and el['lon'] is not None and el['name'] is not None]

https://developers.google.com/maps/documentation/places/web-service/search-along-route

In [29]:

first_point = route_geom.coords[0][1], route_geom.coords[0][0]
m = folium.Map(location=[*first_point], zoom_start=15)

folium.PolyLine(coords_lat_lon, color="blue", weight=2.5, opacity=1).add_to(m)

# marker_cluster = MarkerCluster().add_to(m)

for poi in pois_near_path:
    folium.Marker(
        location=(poi['lat'], poi['lon']),
        popup=f"{poi['name']}\nType: {poi['type']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

m

In [46]:
from src.data_aquisition.wikipedia import fetch_wikipedia_article
import src.processing.text_translation as translate

In [ ]:
poi_names = [(poi['name']) for poi in pois_near_path]
poi_names_en = [translate.detect_and_translate_to_english(poi) for poi in poi_names]
poi_names_lang = [translate.detect_language(poi) for poi in poi_names]

get wikipedia articles about the POIs. Search in english, if no results, then on local language.

In [52]:
wiki_articles = {}

for i in range(len(poi_names_en)):
    if poi_names_lang[i] == 'en':
        summary = fetch_wikipedia_article(poi_names_en[i], language='en', return_summary=True)
    else:
        summary = fetch_wikipedia_article(poi_names[i], language=poi_names_lang[i], return_summary=True)
        summary = translate.detect_and_translate_to_english(summary)

    wiki_articles[poi_names_en[i]] = summary

In [53]:
wiki_articles

{'Justus': "Justus (died on 10 November between 627 and 631) was the fourth archbishop of Canterbury. Pope Gregory the Great sent Justus from Italy to England on a mission to Christianise the Anglo-Saxons from their native paganism; he probably arrived with the second group of missionaries despatched in 601. Justus became the first bishop of Rochester in 604 and signed a letter to the Irish bishops urging the native Celtic church to adopt the Roman method of calculating the date of Easter. He attended a church council in Paris in 614.\nFollowing the death of King Æthelberht of Kent in 616, Justus was forced to flee to Gaul but was reinstated in his diocese the following year. In 624, he was elevated to Archbishop of Canterbury, overseeing the despatch of missionaries to Northumbria. After his death, he was revered as a saint and had a shrine in St Augustine's Abbey, Canterbury, to which his remains were translated in the 1090s.",
 'Roland': 'Roland (Frank: *Hrōziland) is a legendary 8t

#### 2.2 General info

1. Use street names
2. Town areas
3. Town
4. Country

### 3. Utilize LLM to summarize information

### 4. Generate text-to-speech

1. https://elevenlabs.io/text-to-speech (1$ for 30min + commercial use)
2. aws polly (12months free, 1mil char / month, 5 mil char / month)